# Problem Statement

## MNIST Image Classification

The MNIST dataset is a dataset of 60,000 training and 10,000 test examples of handwritten digits, originally constructed by Yann Lecun, Corinna Cortes, and Christopher J.C. Burges. It is very widely used to check simple methods. There are 10 classes in total ("0" to "9"). This dataset has been extensively studied, and there is a history of methods and feature constructions at https://en.wikipedia.org/wiki/MNIST_database and at the original site, http://yann.lecun.com/exdb/mnist/. You should notice that the best methods perform extremely well.

The http://yann.lecun.com/exdb/mnist/ dataset is stored in an unusual format, described in detail on the page.  You do not have to write your own reader.  A web search should yield solutions for both Python and R.  For Python, https://pypi.org/project/python-mnist/ should work.  For R, there is reader code available at https://stackoverflow.com/questions/21521571/how-to-read-mnist-database-in-r. Please note that if you follow the recommendations in the accepted answer there at https://stackoverflow.com/a/21524980, you must also provide the readBin call with the flag signed=FALSE since the data values are stored as unsigned integers.

The dataset consists of 28 x 28 images. These were originally binary images, but appear to be grey level images as a result of some anti-aliasing. I will ignore mid-grey pixels (there aren't many of them) and call dark pixels "ink pixels", and light pixels "paper pixels"; you can modify the data values with a threshold to specify the distinction, as described here https://en.wikipedia.org/wiki/Thresholding_(image_processing). The digit has been centered in the image by centering the center of gravity of the image pixels, but as mentioned on the original site, this is probably not ideal. Here are some options for re-centering the digits that I will refer to in the exercises.

Untouched: Do not re-center the digits, but use the images as is.
Bounding box: Construct a 20 x 20 bounding box so that the horizontal (resp. vertical) range of ink pixels is centered in the box.
Stretched bounding box: Construct a 20 x 20 bounding box so that the horizontal (resp. vertical) range of ink pixels runs the full horizontal (resp. vertical) range of the box. Obtaining this representation will involve rescaling image pixels: you find the horizontal and vertical ink range, cut that out of the original image, then resize the result to 20 x 20. Once the image has been re-centered, you can compute features.

### Question Part A: MNIST using naive Bayes

Model each class of the dataset using a Normal distribution and (separately) a Bernoulli distribution for both untouched images v. stretched bounding boxes, using 20 x 20 for your bounding box dimension.  This should result in 4 total models.  Use the training set to calculate the distribution parameters.

You must write the naive Bayes prediction code.  The distribution parameters can be calculated manually or via libraries.  Additionally, we recommend using a library to load the MNIST data (e.g. python-mnist or scikit-learn) and to rescale the images (e.g. openCV).

Compute the accuracy values for the four combinations of Normal v. Bernoulli distributions for both untouched images v. stretched bounding boxes.  Both the training and test set accuracy will be reported.
For each digit, plot the mean pixel values calculated for the Normal distribution of the untouched images.  In Python, a library such as matplotlib should prove useful.

### Question Part B: MNIST using Decision Forest

Classify MNIST using a decision forest.
For your forest construction, you should investigate four cases. Your cases are: number of trees = (10, 30) X maximum depth = (4, 16). You should compute your accuracy for each of the following cases: untouched raw pixels; stretched bounding box. This yields a total of 8 slightly different classifiers. Please use 20 x 20 for your bounding box dimensions.

You should use a decision forest library.  No need to write your own.

### Code: Part A

In [16]:
# Code: Homework #1 Problem #2A

import numpy as np
import pandas as pd
from datetime import datetime
from scipy.stats import norm
from scipy.stats import multivariate_normal as mvn
from scipy.stats import bernoulli as brn
import random
from scipy.misc import imresize

def get_original_data(file):
    df = pd.read_csv(file,header=None)
    data = df.as_matrix()
    np.random.shuffle(data)
    X = threshold_images(data[:,1:])
    Y = data[:,0]
    return X,Y

def get_cropped_data(file):
    df = pd.read_csv(file,header=None)
    data = df.as_matrix()
    np.random.shuffle(data)
    X = threshold_images(crop_images(data[:,1:]))
    Y = data[:,0]
    return X,Y

def crop_images(X_train):
    a= int(X_train.shape[0])
    X_train_crop = np.zeros(shape= (a,400))
    
    for i in range(len(X_train)):
      image = X_train[i].reshape(28,28)
      row= np.unique(np.nonzero(image)[0])
      col = np.unique(np.nonzero(image)[1])
      image2 = image[min(row):max(row), min(col):max(col)]
      value = imresize(image2, (20, 20))
      X_train_crop[i] =  value.reshape(1,400)
    return X_train_crop

def threshold_images(X_train):
    for i in range(len(X_train)):
      for j in range(len(X_train[i])):
        if X_train[i][j]<128:
          X_train[i][j]=0
        else:
          X_train[i][j]=1
    return X_train

class NaiveBayes(object):
    
    def fit(self, X, Y, smoothing=10e-3):
        self.gaussians = dict()
        self.priors = dict()
        labels = set(Y)
        for c in labels:
            current_x = X[Y == c]
            self.gaussians[c] = {
                'mean': current_x.mean(axis=0),
                'var': current_x.var(axis=0) + smoothing,
            }    
            self.priors[c] = float(len(Y[Y==c])) / len(Y)
    
    def score(self, X, Y):
        P_norm, P_brn = self.predict(X)
        return np.mean(P_norm==Y), np.mean(P_brn==Y)
    
    def predict(self, X):
        N, D = X.shape
        K = len(self.gaussians)
        P_norm = np.zeros((N, K))
        P_brn = np.zeros((N, K))
        for c,g in self.gaussians.items():
            mean, var = g['mean'], g['var']
            P_norm[:,c] = mvn.logpdf(X, mean=mean, cov=var) + np.log(self.priors[c])
            P_brn[:,c] = brn.logpmf(X, mean).sum(axis=1) + np.log(self.priors[c])
                    
        return np.argmax(P_norm, axis=1), np.argmax(P_brn, axis=1) 
    
    
if __name__=='__main__':
    
    #Dataset for Test accuracy
    X_train, y_train = get_original_data('mnist_train.csv')
    X_test, y_test = get_original_data('mnist_test.csv')
    
    X_train_cropped, y_train_cropped = get_cropped_data('mnist_train.csv')
    X_test_cropped, y_test_cropped = get_cropped_data('mnist_test.csv')
    
    #Dataset for training accuracy
    Ntrain = int(len(y_train)*0.8)
    
    Xtrain, Ytrain = X_train[:Ntrain], y_train[:Ntrain]
    Xtest, Ytest = X_train[Ntrain: ], y_train[Ntrain:]
    
    Xtrain_cropped, Ytrain_cropped = X_train_cropped[:Ntrain], y_train_cropped[:Ntrain]
    Xtest_cropped, Ytest_cropped = X_train_cropped[Ntrain: ], y_train_cropped[Ntrain:]
    
    #Models
    model = NaiveBayes()
    model.fit(Xtrain, Ytrain)
    model.norm_score, model.brn_score = model.score(Xtest,Ytest)
    print(f'Train accuracy for untouched images using Normal distribution: {model.norm_score}')
    print(f'Train accuracy for untouched images using Bernoulli distribution: {model.brn_score}')
    
    model_2 = NaiveBayes()
    model_2.fit(X_train, y_train)
    model_2.norm_score, model_2.brn_score = model_2.score(X_test,y_test)
    print(f'Test accuracy for untouched images using Normal distribution: {model_2.norm_score}')
    print(f'Test accuracy for untouched images using Bernoulli distribution: {model_2.brn_score}')
    
    model_3 = NaiveBayes()
    model_3.fit(Xtrain_cropped, Ytrain_cropped)
    model_3.norm_score, model_3.brn_score = model_3.score(Xtest_cropped,Ytest_cropped)
    print(f'Train accuracy for cropped images using Normal distribution: {model_3.norm_score}')
    print(f'Train accuracy for cropped images using Bernoulli distribution: {model_3.brn_score}')
        
    model_4 = NaiveBayes()
    model_4.fit(X_train_cropped, y_train_cropped)
    model_4.norm_score, model_4.brn_score = model_4.score(Xtest_cropped,Ytest_cropped)
    print(f'Test accuracy for cropped images using Normal distribution: {model_4.norm_score}')
    print(f'Test accuracy for cropped images using Bernoulli distribution: {model_4.brn_score}')
    
     


C:\Users\sayan\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if sys.path[0] == '':
C:\Users\sayan\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
C:\Users\sayan\Anaconda3\lib\site-packages\ipykernel_launcher.py:35: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


Train accuracy for untouched images using Normal distribution: 0.793
Train accuracy for untouched images using Bernoulli distribution: 0.837
Test accuracy for untouched images using Normal distribution: 0.7986
Test accuracy for untouched images using Bernoulli distribution: 0.8434
Train accuracy for cropped images using Normal distribution: 0.8105
Train accuracy for cropped images using Bernoulli distribution: 0.817
Test accuracy for cropped images using Normal distribution: 0.8105833333333333
Test accuracy for cropped images using Bernoulli distribution: 0.8183333333333334
